In [2]:
import numpy as np
import pandas as pd
from numpy_financial import irr

# Feasibility Analysis of a wind pumped hydroelectric storage system on the Island of Brava

# Load data
VentoCurvaPotencia = np.loadtxt('VentoCurvaPotencia.txt')
#Pd = pd.read_excel('Brava_consumo.xlsx', skiprows=2, usecols=range(8, 28))  # Assuming the data starts from row 3
# Carregue o arquivo Excel
xl = pd.ExcelFile('Brava_consumo.xlsx')
#Pd = pd.read_excel('Brava_consumo.xlsx', sheet_name='Carga2010-2037')
# Leia os dados da aba "Carga2010-2037"
Pd = pd.read_excel(xl, sheet_name='Carga2010-2037', dtype=float,nrows=8762)
# Wind Turbine Groups: [Power, Quantity, Corresponding column, Reservoir volume]
Cen = np.array([
    [150, 2, 1, 1500],
    [300, 1, 2, 1500],
    [150, 3, 1, 2250],
    [225, 2, 2, 2250],
    [225, 3, 3, 3400],
    [300, 2, 2, 3000],
    [300, 3, 2, 4500]
])

Res = np.zeros((len(Cen), 14))
Res2 = np.zeros((len(Cen), 8))

for k in range(len(Cen)):
    ano = 20
    RS = Cen[k, 3]
    ERmax = RS / 3600 * 9.8 * 110
    h = 8760
    Er = np.zeros((h, ano))
    Erj = ERmax / 2
    Pt = np.zeros((h, ano))
    Pb = np.zeros((h, ano))
    Erej = np.zeros((h, ano))
    Pge = np.zeros((h, ano))
    Pwr = np.zeros((h, ano))
    nb = 0.86
    nt = 0.86
    nw = 0.97
    Te = Cen[k, 0]
    ne = Cen[k, 1]
    PDmin = 256 * 0.5

    Pwt = ne * nw * VentoCurvaPotencia[:, int(Cen[k, 2]) - 1]

    Ptmax = np.mean(Pwt)

    for i in range(ano):
        for j in range(h):
            if Pwt[j] > Pd.iloc[j, i] * 0.35:
                Pwr[j, i] = Pd.iloc[j, i] * 0.35

                if (Pwt[j] - Pwr[j, i]) * nb < (ERmax - Erj):
                    Pb[j, i] = Pwt[j] - Pwr[j, i]
                    Erj += (Pwt[j] - Pwr[j, i]) * nb
                else:
                    Pb[j, i] = (ERmax - Erj) / nb
                    Erj = ERmax
                    Erej[j, i] = (Pwt[j] - Pwr[j, i]) - (ERmax - Erj) / nb
            else:
                Pwr[j, i] = Pwt[j]

                

            if (Pd.iloc[j, i] - Pwr[j, i]) <= Erj * nt:
                if (Pd.iloc[j, i] - Pwr[j, i]) <= Ptmax:
                    Pt[j, i] = Pd.iloc[j, i] - Pwr[j, i]
                    Erj -= Pt[j, i] / nt
                elif Ptmax < (Pd.iloc[j, i] - Pwr[j, i]) <= (Ptmax + PDmin):
                    Pge[j, i] = PDmin
                    Pt[j, i] = (Pd.iloc[j, i] - Pwr[j, i]) - Pge[j, i]
                    Erj -= Pt[j, i] / nt
                elif (Pd.iloc[j, i] - Pwr[j, i]) > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj -= Ptmax / nt
            elif (Pd.iloc[j, i] - Pwr[j, i]) > Erj * nt:
                if Erj * nt > Ptmax and Erj * nt < (Ptmax + PDmin):
                    if Ptmax >= PDmin and (Pd.iloc[j, i] - Pwr[j, i]) <= Ptmax:
                        Pge[j, i] = PDmin
                        Pt[j, i] = (Pd.iloc[j, i] - Pwr[j, i]) - PDmin
                        Erj -= Pt[j, i] / nt
                    else:
                        Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i]
                elif Erj * nt > (Ptmax + PDmin):
                    Pt[j, i] = Ptmax
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i] - Pt[j, i]
                    Erj -= Ptmax / nt
                else:
                    Pge[j, i] = Pd.iloc[j, i] - Pwr[j, i]

            Er[j, i] = Erj

#-----------------------
  # --------------------------    Resultados    --------------------------------

# pW = percentual de eólica na rede
pW = np.sum(Pwr) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia eolica na rede = %3.2f' % pW)

# pH = percentual de energia hídrica na rede elétrica
pH = np.sum(Pt) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia hidrica na rede = %3.2f' % pH)

# pER = percentual de penetração total de Renováveis (eólica+hidro)
pER = (np.sum(Pwr + Pt) / np.sum(Pd.to_numpy())) * 100
print('Total de RE = %3.2f' % pER)

# ERt = energia renovável aproveitada (eolica injetada diretamente e hidraulica), kWh
ERt = np.sum(Pwr + Pt)

# pDG = percentual de energia fossil na rede
pDG = np.sum(Pge) / np.sum(Pd.to_numpy()) * 100
print('Percentual de Energia fossil na rede = %3.2f' % pDG)

# pdiss = percentual de energia dissipada devido a reservatorio cheio
pdiss = np.sum(Erej) / (np.sum(Pwt) * ano) * 100
print('Percentual de  Energia dissipada devido a reservatorio cheio: %3.2f' % pdiss)

# Análise Econômica e Financeira
EeC = 56554.48  # estudos e concursos
CTe = 2114.14 * Te * ne  # custo dos aerogeradores
CTh = 1469.38 * np.max(Pt)  # custo das turbinas hidráulicas
CBb = 1057.95 * np.max(Pb)  # custo das bombas
CRes = 57.4 * RS  # custo do reservatório
CTub = 88.25 * 1.3 * 2 * 250  # custo das condutas
Ctot = (EeC + CTe + CTh + CBb + CRes + CTub) * 1.15  # custo total (15% para custos diversos)
# Dados
Pdiesel = 1.17  # Preço do litro do gasóleo
# Pdiesel = 112.2  # Preço do litro do gasóleo (descomente se necessário)
CP = -0.3 * Ctot  # Empréstimo de 70% do valor do investimento
CPa = -0.3 * Ctot  # Inicializa o cash flow atualizado acumulado
r = 0.04  # Juros de 4,0%/ano
Tax = 0.14  # Taxa de atualização
t = 15  # Anos de empréstimo bancário
# Matriz de avaliação financeira - AF
AF = np.zeros((9, ano))

for i in range(ano):
    AF[0, i] = (np.sum(Pwr[:, i]) + np.sum(Pt[:, i])) * 0.212 / 0.84 * Pdiesel
    AF[1, i] = np.sum(Pwt) * 0.024 + np.sum(Pb[:, 0]) * 0.082

    if i < 15:
        AF[2, i] = Ctot / 15
    else:
        AF[2, i] = 0

    if i < t:
        AF[3, i] = 0.7 * Ctot / t + 0.7 * (Ctot - Ctot / t * (i - 1)) * r
    else:
        AF[3, i] = 0

    AF[4, i] = AF[0, i] - AF[1, i] - AF[2, i] - AF[3, i]

    if AF[4, i] > 0:
        AF[5, i] = AF[4, i] * 0.75
    else:
        AF[5, i] = AF[4, i]

    AF[6, i] = AF[5, i] + AF[2, i]
    AF[7, i] = AF[6, i] / (1 + Tax) ** i
    AF[8, i] = CPa + AF[7, i]

    CPa = AF[8, i]

# Imprimindo resultados
print('Energia eolica aproveitada: %10.2f' % ((np.sum(Pwr) + np.sum(Pt)) / (np.sum(Pwt) * ano) * 100))
print('Toneladas metricas de gasoleo poupados: %5.2f' % ((np.sum(Pwr) + np.sum(Pt)) * 0.212 / 0.84 / 1000))
print('mESC poupados em gasoleo: %10.2f' % ((np.sum(Pwr) + np.sum(Pt)) * 0.212 / 0.84 * 0.1122))

#-------------

# PAY-BACK
pback = None  # inicializa pback com 'None'
for i in range(ano - 1):
    if AF[8, i] * AF[8, i + 1] <= 0:
        pback = (-AF[8, i] * (i + 1) + i * AF[8, i + 1]) / (AF[8, i + 1] - AF[8, i])
        print('Pay-back = %2.1f' % pback)
    else:
        pass

# NPV
NPV = AF[8, ano - 1]
print('Net Present Value = %9.2f Esc' % NPV)

# IRR ou Taxa Interna de Retorno (TIR)
TIR = npf.irr(np.concatenate(([CP], AF[6, 0:ano])))
print('TIR (ou IRR) = %9.3f' % TIR)

# Levelized Cost of Energy - LCOE
LCOE = np.sum((AF[1, :] + AF[3, :]) / ((1 + Tax) ** np.arange(1, ano + 1))) / (
            np.sum(ERt / ((1 + Tax) ** np.arange(1, ano + 1))))
print('LCOE = %9.3f' % LCOE)

# Tabelas de resultados
# Não foi encontrado um equivalente direto para a criação de tabelas como em MATLAB no Python padrão.
# Dependendo do ambiente de execução, pode ser necessário usar bibliotecas adicionais como pandas para criar tabelas.



KeyboardInterrupt: 